In [ ]:
import os
import sys
import csv
import yaml
import numpy as np
import open3d as o3d
from rosbag import Bag

# Merge Multiple ROSbag Files

In [ ]:

def merge():
    input_folder = "xxxxxx"
    output_folder = "xxxxxxx"
    outbag_name = "xxxxx"
    input_bags = os.listdir(input_folder)
    input_bags.sort()  # 根据文件名进行排序
    print("Writing bag file: " + outbag_name)

    with Bag(os.path.join(output_folder, outbag_name), 'w') as ob:
        for ifile in input_bags:
            print("> Reading bag file: " + ifile)
            with Bag(os.path.join(input_folder, ifile), 'r') as ib:
                for topic, msg, t in ib:
                    ob.write(topic, msg, t)
merge()


# Convert Pose Format

In [ ]:
# 该文件的作用是将csv格式转换为标准的tum位姿格式

input_file = './dataset/SubT_MRS/SubT_MRS_Urban_Challenge_UGV2/poses/ground_truth_path.csv'
output_file = './dataset/SubT_MRS/SubT_MRS_Urban_Challenge_UGV2/poses/gt_poses_tum.txt'

with open(input_file, 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  # Skip the header
    with open(output_file, 'w') as outfile:
        for row in reader:
            nsec, x, y, z, qx, qy, qz, qw = map(float, row)
            sec = nsec * 1e-9  # Convert nanoseconds to seconds
            output_line = f"{sec} {x} {y} {z} {qx} {qy} {qz} {qw}\n"
            outfile.write(output_line)

print("Conversion completed, file saved as", output_file)



# Mapping Performance Evaluation


In [ ]:
def quaternion_to_rotation_matrix(qx, qy, qz, qw):
    """
    将四元数转换为旋转矩阵。

    输入:
    - qx, qy, qz, qw: 四元数的分量

    返回:
    - rotation_matrix: 3x3的旋转矩阵
    """
    # 四元数的归一化
    norm = np.sqrt(qx * qx + qy * qy + qz * qz + qw * qw)
    qx /= norm
    qy /= norm
    qz /= norm
    qw /= norm

    # 计算旋转矩阵
    rotation_matrix = np.array([
        [1 - 2 * (qy * qy + qz * qz), 2 * (qx * qy - qz * qw), 2 * (qx * qz + qy * qw)],
        [2 * (qx * qy + qz * qw), 1 - 2 * (qx * qx + qz * qz), 2 * (qy * qz - qx * qw)],
        [2 * (qx * qz - qy * qw), 2 * (qy * qz + qx * qw), 1 - 2 * (qx * qx + qy * qy)]
    ])

    return rotation_matrix


## Newer College Dataset Ncd Sequence
#%%
# ImMesh
matrix_values = [
    5.304626993818075675e-01, -8.474622417882305969e-01, 2.042261633276983360e-02, -8.377865843928848644e-02,
    8.463450710843981595e-01, 5.308216667107832354e-01, 4.391332211528171242e-02, 3.370663104058911230e+00,
    -4.805564502133059107e-02, -6.009799083416286596e-03, 9.988265833638158009e-01, 7.037440120229881968e-01
]
T_lidar = np.vstack([np.array(matrix_values).reshape(3, 4), [0, 0, 0, 1]])

T_lidar_imu = np.array([[-1.0, 0, 0, -0.006253],
                        [0, -1.0, 0, 0.011775],
                        [0, 0, 1.0, -0.028535],
                        [0, 0, 0, 1]])


## Newer College Dataset Extension Math Easy Sequence

In [ ]:
T_lidar = np.eye(4)

# SLAMesh
T_lidar[:3, :3] = quaternion_to_rotation_matrix(-0.00987445, 0.00774057, 0.842868, 0.537974)
T_lidar[:3, 3] = np.array([-23.7176, -31.2646, 1.03258])

# ImMesh
# T_lidar[:3, :3] = quaternion_to_rotation_matrix(-0.00248205, 0.00444627, 0.842838, 0.538143)
# T_lidar[:3, 3] = np.array([-23.7202, -31.2861, 1.04326])

T_lidar_imu = np.array([
    [1.0, 0, 0, 0.014],
    [0, 1.0, 0, -0.012],
    [0, 0, 1.0, -0.015],
    [0, 0, 0, 1.0]])

T = T_lidar @ T_lidar_imu

## Transform Mesh To Align The Ground Truth

In [ ]:
def mesh_transform():
    mesh_file = "./math_easy.ply"
    output_file = "./math_easy_transformed.ply"

    if not os.path.exists(mesh_file):
        sys.exit(f"Mesh file {mesh_file} does not exist.")
    print("Loading Mesh file: ", mesh_file)

    # Load Mesh
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()
    transformation_matrix = np.array([[-4.20791735e-01, -9.07157072e-01,  6.01526210e-04, -2.37152142e+01],
                                      [ 9.07112929e-01, -4.20764518e-01,  1.01663692e-02, -3.12685037e+01],
                                      [-8.96939285e-03,  4.82357635e-03,  9.99948140e-01,  1.02807732e+00],
                                      [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

    mesh.transform(transformation_matrix)
    o3d.io.write_triangle_mesh(output_file, mesh)

mesh_transform()

# Visualize Mesh


In [ ]:
def vis_mesh():
    mesh_file = "./ours_mesh_20cm.ply"  # the path of the mesh which you want to visualize
    if not os.path.exists(mesh_file):
        sys.exit(f"Mesh file {mesh_file} does not exist.")
    print("Loading Mesh file: ", mesh_file)

    # Load the mesh
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()
    # Check if the mesh was loaded successfully
    if not mesh.has_vertices():
        sys.exit("Failed to load the mesh. No vertices found.")
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Mesh Visualization")
    vis.add_geometry(mesh)
    opt = vis.get_render_option()
    opt.light_on = True  # Enable lighting to show the mesh color
    opt.mesh_show_back_face = True
    # Enable shortcuts in the console (e.g., Ctrl+9)
    vis.run()
    vis.destroy_window()

vis_mesh()